## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-13-15-49-59 +0000,nyt,"Juan Ponce Enrile, a Political Power in the Ph...",https://www.nytimes.com/2025/11/13/world/asia/...
1,2025-11-13-15-48-40 +0000,nyt,Epstein Bantered Regularly With Larry Summers,https://www.nytimes.com/2025/11/12/us/politics...
2,2025-11-13-15-38-23 +0000,nyt,Dozens Are Hospitalized After Ammonia Leak in ...,https://www.nytimes.com/2025/11/13/us/weatherf...
3,2025-11-13-15-33-30 +0000,nypost,Crazed boyfriend hacked up partner with chains...,https://nypost.com/2025/11/13/us-news/crazed-b...
4,2025-11-13-15-32-43 +0000,nyt,"As Shutdown Ends, When Will SNAP and Air Trave...",https://www.nytimes.com/2025/11/12/us/shutdown...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,44
7,epstein,29
28,shutdown,23
312,government,19
44,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
204,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...,174
155,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...,132
200,2025-11-13-01-27-32 +0000,nypost,"Trump signs funding bill to end historic, 43-d...",https://nypost.com/2025/11/12/us-news/house-vo...,125
171,2025-11-13-04-11-30 +0000,wapo,Longest government shutdown in U.S. history en...,https://www.washingtonpost.com/business/2025/1...,121
33,2025-11-13-13-58-49 +0000,nypost,Trump-obsessed writer Michael Wolff encouraged...,https://nypost.com/2025/11/13/us-news/michael-...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
204,174,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...
155,88,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...
105,50,2025-11-13-10-30-00 +0000,wsj,Heard on the Street: Borrowing from a builder ...,https://www.wsj.com/economy/housing/builders-c...
29,43,2025-11-13-14-08-09 +0000,nypost,Middle school teacher allegedly had sex with s...,https://nypost.com/2025/11/13/us-news/pervy-te...
171,35,2025-11-13-04-11-30 +0000,wapo,Longest government shutdown in U.S. history en...,https://www.washingtonpost.com/business/2025/1...
337,35,2025-11-12-16-45-03 +0000,wapo,Ukraine’s justice and energy ministers resign ...,https://www.washingtonpost.com/world/2025/11/1...
258,33,2025-11-12-21-19-00 +0000,wsj,Government reports on inflation and the labor ...,https://www.wsj.com/politics/policy/white-hous...
281,33,2025-11-12-20-09-00 +0000,wsj,Atlanta Fed President Raphael Bostic will reti...,https://www.wsj.com/economy/central-banking/at...
137,33,2025-11-13-09-00-00 +0000,cbc,"After 2 years of war, Gaza electric company ta...",https://www.cbc.ca/news/world/gaza-power-elect...
201,32,2025-11-13-01-17-00 +0000,wsj,"Rep. Adelita Grijalva (D., Ariz.) became the c...",https://www.wsj.com/politics/policy/petition-f...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
